# Staging OpenAI API


In [5]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

In [6]:
user_input = "Hello, suggest a trip plan in Tokyo"

messages = [
    {"role": "system", "content": "Only suggest 1 activity for the trip plane"},
    {"role": "user", "content": user_input},
]

completion = client.chat.completions.create(model="gpt-4o-mini", messages=messages)
re_json = completion.choices[0].message.content
print(re_json)

One great activity for your trip to Tokyo is to visit the historic Asakusa district. You can explore Senso-ji, Tokyo's oldest temple, and stroll through the bustling Nakamise Street, where you'll find traditional snacks and souvenirs. Enjoy the vibrant atmosphere, and don't forget to try some freshly made melon bread or ningyo-yaki! Afterward, take a relaxing walk along the Sumida River or enjoy a scenic boat ride for a different perspective of the city. This combination of culture, food, and scenic views makes for a memorable experience in Tokyo.


# GMAIL API


In [ ]:
# GMAIL
import base64


# GPT written, just decodes gmail body content data
def decode_base64_data(data_str):
    missing_padding = len(data_str) % 4
    if missing_padding:
        data_str += "=" * (4 - missing_padding)
    decoded_bytes = base64.urlsafe_b64decode(data_str.encode("utf-8"))
    return decoded_bytes.decode("utf-8", errors="replace")


import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build


def get_credentials(token_path="token.json", scopes=None):
    """
    Gets credentials from user
    Args:
        token_path: (str) the token.json file unique to local users
        scopes: (str) sets scope for get_credentials
    """
    if scopes is None:
        scopes = ["https://www.googleapis.com/auth/gmail.readonly"]
    creds = None
    if os.path.exists(token_path):
        creds = Credentials.from_authorized_user_file(token_path, scopes)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            client_config = {
                "installed": {
                    "client_id": os.environ.get("GOOGLE_CLIENT_ID_DESK"),
                    "client_secret": os.environ.get("GOOGLE_CLIENT_SECRET_DESK"),
                    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
                    "token_uri": "https://oauth2.googleapis.com/token",
                }
            }
            flow = InstalledAppFlow.from_client_config(client_config, scopes)
            creds = flow.run_local_server(port=0)
        with open(token_path, "w") as token_file:
            token_file.write(creds.to_json())
    return creds


def build_gmail_service(creds):
    return build("gmail", "v1", credentials=creds)


# gets credentials
creds = get_credentials()

# build the gmail service
service = build_gmail_service(creds)

# Id's for messages
results = (
    service.users().messages().list(userId="me", q="is:unread").execute()["messages"]
)

# using getting the top 10 messages
top_k_messages = results[:10] if len(results) > 10 else results

# getting the each message with ids
message_ids = [message.get("id") for message in top_k_messages]

# getting the resulting messages
res_messages = [
    service.users()
    .messages()
    .get(userId="me", id=id, format="full")
    .execute()
    .get("payload")
    for id in message_ids
]

{'body': {'size': 0},
 'filename': '',
 'headers': [{'name': 'Delivered-To', 'value': 'contact@vizcom.co'},
             {'name': 'Received',
              'value': 'by 2002:a05:7022:4199:b0:9a:c19:4c0e with SMTP id '
                       'i25csp4903024dli;        Tue, 27 May 2025 11:39:08 '
                       '-0700 (PDT)'},
             {'name': 'X-Received',
              'value': 'by 2002:a05:6e02:2482:b0:3d3:db70:b585 with SMTP id '
                       'e9e14a558f8ab-3dc9b72443emr160912575ab.21.1748371148703;        '
                       'Tue, 27 May 2025 11:39:08 -0700 (PDT)'},
             {'name': 'ARC-Seal',
              'value': 'i=1; a=rsa-sha256; t=1748371148; cv=none;        '
                       'd=google.com; s=arc-20240605;        '
                       'b=isynD+b6SuUAWfDbkolPg0eyOZRvXQKM5qggr55/5pBe4lK3FavGBfPrtbaI73687r         '
                       'xX5YUNJR4LflVygWUr5JcRZ2D8RmNqQ0Sx3q1PRAghsqLU7NVfHU9Bg/ElH09Av98I1l         '
                   

## OPENAI: No Context Response + BS Confidence Score?


In [73]:
import json
from pprint import pprint

In [27]:
user_input = "What's the difference between detailed sharp and detailed smooth?"
context = None

support_bot = [
    {
        "role": "system",
        "content": f"Answer questions with based only off of given context: {context}. Do so politely and professionally with a consistent formatting, "
        "Hello [sender]\n[body]\nThank You,\nVizcom Team"
        f"here is the question type for better reference {data.type_of_q}. Also provide a confidence score based on how accurate you feel your answer is"
        "Only use the context given and conversation, nothing else. Return low confidence scores if not enough context is given"
        "Return a json containing the response and confidence score. Make sure the score is reasonable. Do nothing else."
        "Here is an example json: '{response: 'text', confidence: float/1.0}' ",
    },
    {"role": "user", "content": user_input},
]

completion = client.chat.completions.create(model="gpt-4o-mini", messages=support_bot)
response = str(completion.choices[0].message.content)

pprint(json.loads(response))

{'confidence': 0.6,
 'response': 'Hello, \n'
             'The difference between detailed sharp and detailed smooth '
             'typically refers to the texture and clarity of images. Detailed '
             'sharp tends to emphasize fine lines and edges, creating a more '
             'pronounced and crisp appearance. In contrast, detailed smooth '
             'may focus on a softer, more blended look while still maintaining '
             "detail. However, without additional context, it's challenging to "
             'provide a more specific answer.\n'
             'Thank You,\n'
             'Vizcom Team'}


# CONTEXT


In [2]:
import re


def load_as_string(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        data = f.read()
        return re.sub(r"\s+", " ", data).strip()

# Enginnering Data?


In [19]:
json_data = load_as_string("context.txt")

# Random Questions

In [20]:
user_input = "Why won't my animation work in Vizcom"
context = json_data

support_bot = [
    {
        "role": "system",
        "content": (
            "You are Vizcom’s professional support assistant. "
            f"Do NOT guess or introduce outside information. Use ONLY the provided context: {context}. \n"
            "Answer politely and with detailed descriptions, but no step by step guides unless requested, link any relevant reference links for the user in your response, \n"
            "following this format:\n\n"
            "Hello [sender]\n"
            "[Your answer here]\n"
            "Thank You,\n"
            "Vizcom Team\n\n"
            "• Return a thought process and always include the reference link(s) if possible (i.e. how you arrived at the answer and where you found it).\n\n"
            "Return ONLY a JSON object with keys:\n"
            "  • response (string)\n"
            "  • reference link (string)\n"
            "  • thought_process (string)\n\n"
            "Example:\n"
            '{"response": "…", "reference_link": "…", "thought_process": "…"}\n'
        ),
    },
    {"role": "user", "content": user_input},
]

completion = client.chat.completions.create(model="gpt-4o-mini", messages=support_bot)
response = str(completion.choices[0].message.content)
try:
    {pprint(json.loads(response))}
except:
    {print(response)}

{"response": "There may be several reasons why your animation is not working in Vizcom. First, ensure that you have one or more existing models or sketches in your workbench as they are prerequisites for using the animation tool. Additionally, check that you have accessed the Animate mode correctly by clicking the workbench button and selecting the Animate feature. If there were any issues in connecting your design to the animation prompt window, or if the prompt you wrote encountered errors, it might prevent the animation from generating correctly. Additionally, try selecting between Turbo Mode for speed or Standard Mode for better adherence to the prompt. If these steps are all correct and it still doesn’t work, consider checking for any updates or issues with your internet connection. \n\nFor more details, you can refer to the how-to animate documentation here: https://docs.vizcom.ai/how-to-animate.", "reference_link": "https://docs.vizcom.ai/how-to-animate", "thought_process": "The

In [169]:
checker_bot = [
    {
        "role": "system",
        "content": (
            f"Your only job is to use the given context and asses how confident you are of the answer given by the user with respect to the question: {user_input}. Use ONLY the provided context: {context}. "
            "• A confidence score between 0.00 and 1.00 calculated using the below metrics.\n"
            "\t• How relevant the question is to the given context above?\n"
            "\t• Was the question answered or deferred to documentation?\n"
            "\t• How detailed were we?\n"
            "\t• Was the thought process logical and valid?\n"
            "Return ONLY a JSON object with keys:\n"
            "  • confidence (float)\n\n"
            "Example:\n"
            '{"confidence": …}\n'
            "If you feel that there needs to be more information provided, confidence should be 0.00"
        ),
    },
    {"role": "user", "content": response},
]

completion = client.chat.completions.create(model="gpt-4o-mini", messages=checker_bot)
confidence = str(completion.choices[0].message.content)
try:
    {pprint(json.loads(confidence))}
except:
    {print(confidence)}

{'confidence': 0.85}
